# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
# Импортируем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
# Создаем датафрейм с исходными данными
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# Датафрейм уже приведен в приемлимый вид. Пропусков нет, форматы данных соответствующие
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# Выводим информацию
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Итого в датафрейме содержится следующая информация
<ol>сalls — количество звонков,</ol>
<ol>minutes — суммарная длительность звонков в минутах,</ol>
<ol>messages — количество sms-сообщений,</ol>
<ol>mb_used — израсходованный интернет-трафик в Мб,</ol>
<ol>is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).</ol>

## Разбейте данные на выборки

In [5]:
# Разобьем данные на выборки
train, test = train_test_split(df, test_size=0.20, random_state=42)
# 20% выборки оставляем на тест

In [6]:
# разделяем тестовые данные на признаки и результативную переменную
feature_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

In [7]:
# Выделим из train валидационную выборку
df_train, df_valid = train_test_split(train, test_size=0.25, random_state=42)
features_val = df_valid.drop(['is_ultra'], axis=1)
features_train =   df_train.drop(['is_ultra'], axis=1)
target_val = df_valid['is_ultra']
target_train =  df_train['is_ultra']

In [8]:
#добавлено ревьюером
features_train.shape, features_val.shape, feature_test.shape

((1928, 4), (643, 4), (643, 4))

In [9]:
# Исправила разделение на выборки

## Исследуйте модели

Создадим три модели - решающее дерево, случайный лес и логистическую регрессию. Для этого попробуем попереберать гиперпараметры этих моделей

In [10]:
# Решающее дерево
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(features_train, target_train)
    accuracy = model.score(features_val, target_val)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "    Качество:", best_accuracy)

Глубина дерева: 7     Качество: 0.7884914463452566


In [11]:
# Случайный лес

best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    for est in range(5,50,5):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=42)
        model.fit(features_train, target_train)
        accuracy = model.score(features_val, target_val)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est,  "  Качество:", best_accuracy)

Глубина дерева: 4     Количество деревьев: 20   Качество: 0.8040435458786936


In [12]:
# Логистическая регрессия
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_val, target_val)
print("Качество:", accuracy)

Качество: 0.7216174183514774


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Проверьте модель на тестовой выборке

In [13]:
# последовательно проверяем модели на тестовой выборке
# Логистическая регрессия 
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(feature_test, target_test)
print("Качество:", accuracy)

Качество: 0.702954898911353


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
# Дерево решений
model = DecisionTreeClassifier(max_depth=7, random_state=42)
model.fit(features_train, target_train)
accuracy = model.score(feature_test, target_test)
print("Качество:", accuracy)

Качество: 0.7993779160186625


In [15]:
# Случайный лес
model = RandomForestClassifier(max_depth=4, n_estimators=20, random_state=42)
model.fit(features_train, target_train)
accuracy = model.score(feature_test, target_test)
print("Качество:", accuracy)

Качество: 0.8118195956454122


Наилучшее качество модели показывает случайный лес с глбиной дерева 4 и числом деревьев 20 - качество - 0.8118

## (бонус) Проверьте модели на адекватность

Проверим модель на адекватность с помощью сравнения с модели с простым классиикатором - DummyClassifier

In [16]:
model = DummyClassifier(random_state=412)
model.fit(features_train, target_train)
acc = model.score(feature_test, target_test)
print("Качество простого прогноза :", acc)
print("Качество прогноза дерева :", accuracy)

Качество простого прогноза : 0.567651632970451
Качество прогноза дерева : 0.8118195956454122


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
